In [4]:
field_easy = [  # 0 = empty, 1 = food, 2 = wall, 3 = pacman
    [3, 1, 1, 1],
    [0, 2, 2, 1],
    [0, 1, 2, 1],
    [0, 0, 0, 1],
]

field_medium = [
    [3, 1, 1, 2, 1],
    [0, 2, 1, 2, 1],
    [1, 2, 1, 1, 1],
    [1, 1, 2, 2, 1],
    [0, 1, 1, 1, 1],
]

field_hard = [
    [3, 2, 1, 1, 2, 1],
    [1, 2, 0, 2, 2, 1],
    [1, 1, 1, 0, 1, 1],
    [2, 2, 1, 2, 1, 2],
    [1, 0, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 1],
]

field_very_hard = [
    [3, 2, 1, 2, 1, 2, 1],
    [1, 2, 0, 2, 0, 2, 1],
    [1, 1, 1, 1, 1, 2, 1],
    [2, 2, 2, 2, 1, 2, 1],
    [1, 0, 0, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 1],
]

fields = [field_easy, field_medium, field_hard, field_very_hard]

# Definição de classes

In [ ]:
class PacMan:
    def __init__(self, field, x_pos=0, y_pos=0, direction='right'):
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.direction = direction
        self.field = field

    def move(self):
        self.field[self.y_pos][self.x_pos] = 0

        if self.direction == 'right' and self.x_pos + 1 < len(self.field[0]):
            if self.field[self.y_pos][self.x_pos + 1] != 2:
                self.x_pos += 1
        elif self.direction == 'left' and self.x_pos - 1 >= 0:
            if self.field[self.y_pos][self.x_pos - 1] != 2:
                self.x_pos -= 1
        elif self.direction == 'up' and self.y_pos - 1 >= 0:
            if self.field[self.y_pos - 1][self.x_pos] != 2:
                self.y_pos -= 1
        elif self.direction == 'down' and self.y_pos + 1 < len(self.field):
            if self.field[self.y_pos + 1][self.x_pos] != 2:
                self.y_pos += 1

        has_food = self.field[self.y_pos][self.x_pos] == 1

        self.field[self.y_pos][self.x_pos] = 3

        return has_food

    def __str__(self):
        return f"PacMan is at ({self.x_pos}, {self.y_pos})"


class Node:
    def __init__(self, field, father, x_pos, y_pos, depth=0):
        self.depth = depth
        self.field = field
        self.father = father
        self.x_pos = x_pos
        self.y_pos = y_pos

    def generate_next_nodes(self, prune=True):
        next_nodes = []
        for direction in ['right', 'left', 'up', 'down']:
            new_field = [row[:] for row in self.field]
            current_pos = self.x_pos, self.y_pos
            pacman = PacMan(new_field, self.x_pos, self.y_pos, direction)
            has_food = pacman.move()

            if pacman.x_pos == current_pos[0] and pacman.y_pos == current_pos[1]:
                continue

            new_field = pacman.field
            if has_food:
                next_nodes.insert(
                    0, Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
            else:
                next_nodes.append(
                    Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
        return next_nodes

    def generate_backtrack_nodes(self, reference_field, prune=True):
        next_nodes = []
        for direction in ['right', 'left', 'up', 'down']:
            # new_field = [row[:] for row in self.field]
            current_pos = self.x_pos, self.y_pos
            pacman = PacMan(self.field, self.x_pos, self.y_pos, direction)
            pacman.move()

            if prune and pacman.x_pos == current_pos[0] and pacman.y_pos == current_pos[1]:
                continue

            new_field = pacman.field
            if reference_field[current_pos[0]][current_pos[1]] == 1:
                new_field[current_pos[0]][current_pos[1]] = 1

            next_nodes.append(
                Node(new_field, self, pacman.x_pos, pacman.y_pos, self.depth + 1))
        return next_nodes

    def get_state_key(self):
        return str(self.field), self.x_pos, self.y_pos

    def am_i_goal(self):
        for row in self.field:
            if 1 in row:
                return False
        return True

    def distance_to_nearest_food(self):
        queue = [(self.x_pos, self.y_pos, 0)]
        visited = []

        while len(queue) > 0:
            x, y, dist = queue.pop(0)

            already_visited = False
            for vx, vy in visited:
                if vx == x and vy == y:
                    already_visited = True
                    break
            if already_visited:
                continue

            visited.append((x, y))

            if self.field[y][x] == 1:
                return dist

            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nx = x + dx
                ny = y + dy

                if 0 <= nx < len(self.field[0]) and 0 <= ny < len(self.field):
                    if self.field[ny][nx] != 2:
                        queue.append((nx, ny, dist + 1))

        return float('inf')

    def __str__(self):
        return f"({self.x_pos}, {self.y_pos})"

# Testes inicias do PacMan

## Teste unitário de geração de novos nodos

In [38]:
field = [ # 0 = empty, 1 = food, 2 = wall, 3 = pacman
  [3, 1, 1, 1],
  [0, 2, 2, 1],
  [0, 1, 2, 1],
  [0, 0, 0, 1],
]

pacman = PacMan(field, 0, 0, 'right')

node1 = Node(field, None, pacman.x_pos, pacman.y_pos)
next_nodes = node1.generate_next_nodes()

for node in next_nodes:
  print(node)  


Node at (1, 0)
Node at (0, 0)
Node at (0, 0)
Node at (0, 1)


## Teste de movimentação do PacMan

In [ ]:
ok = True
pacman = PacMan(field=field)

while ok:
  print("Current field:")
  for row in field:
    print(row)

  command = input("Enter command (up, down, left, right) or 'exit' to quit: ").strip().lower()

  if command == 'exit':
    ok = False
  elif command in ['up', 'down', 'left', 'right']:
    pacman.direction = command
    pacman.move()
    print(pacman)
  else:
    print("Invalid command. Please try again.")

Current field:
[3, 1, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (1, 0)
Current field:
[0, 3, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (2, 0)
Current field:
[0, 0, 3, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 3]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 0]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]


# Busca em largura

In [3]:
def busca_em_largura(inicial_node, prune=False, debug=False):
    iterations = 0
    current_nodes = [inicial_node]
    visited = []

    while len(current_nodes) > 0:
        this_node = current_nodes.pop(0)

        if prune:
            current_state = (tuple(map(tuple, this_node.field)),
                             this_node.x_pos, this_node.y_pos)
            if current_state in visited:
                continue
            visited.append(current_state)

        if this_node.am_i_goal():
            return this_node, iterations

        next_nodes = this_node.generate_next_nodes()
        current_nodes.extend(next_nodes)

        if debug:
            for node in current_nodes:
                print(node)
            print('---------------')

        iterations += 1

    return None, iterations

In [10]:
def pac_man_com_busca(nodo_final, iterations, debug=False):
  print(f"Number of iterations: {iterations}\n")

  if not nodo_final:
    print("Pacman could not find food")
    print("Iterations: ", iterations)
    return

  # Print todos os nodos pais até o nodo inicial
  nodo_atual = nodo_final
  while nodo_atual.father is not None:
    this_field = nodo_atual.field
    if debug:
      for row in this_field:
        print(row)
      print("\n")
    nodo_atual = nodo_atual.father
  if debug:
    print(nodo_atual) # nodo inicial
    print("Final node reached")
  
  return

## Teste com campo simples sem poda

In [15]:
nodo_final, iterations = busca_em_largura(Node(field_easy, None, 0, 0), False)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 804



## Teste com campo simples com poda

In [11]:
nodo_final, iterations = busca_em_largura(Node(field_easy, None, 0, 0), True)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 73



## Teste com todos os campos com poda

In [ ]:
iterações = []

for index, field in enumerate(fields):
  nodo_final, iterations = busca_em_largura(Node(field, None, 0, 0), True)
  iterações.append(iterations)
  print(f"Field {index + 1}: {iterations} iterations")

Field 1: 73 iterations
Field 2: 1175 iterations
Field 3: 11465 iterations
Field 4: 17126 iterations
Média: 7459.75


# Busca em profundidade

In [17]:
def busca_em_profundidade(initial_node, m, prune=False, debug=False):
    iterations = 0
    current_nodes = [initial_node]
    visited = []

    while len(current_nodes) > 0:
        this_node = current_nodes.pop()

        if prune:
            current_state = (tuple(map(tuple, this_node.field)),
                             this_node.x_pos, this_node.y_pos)
            if current_state in visited:
                continue
            visited.append(current_state)

        if this_node.am_i_goal():
            return this_node, iterations

        if this_node.depth < m:
            next_nodes = this_node.generate_next_nodes(prune=True)
            current_nodes = next_nodes + current_nodes  # Depth-first behavior

        if debug:
            for node in current_nodes:
                print(node)
            print('---------------')

        iterations += 1

    return None, iterations

## Teste com campo simples sem poda

In [20]:
nodo_final, iterations = busca_em_profundidade(Node(field_easy, None, 0, 0), 200, False)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 1880



## Teste com campo simples com poda

In [21]:
nodo_final, iterations = busca_em_profundidade(Node(field_easy, None, 0, 0), 200, True)
pac_man_com_busca(nodo_final, iterations, False)

Number of iterations: 98



## Teste busca em profundidade interativa

In [26]:
for m in range(20):
  nodo_final, iterations = busca_em_profundidade(Node(field_easy, None, 0, 0), m, True)
  if nodo_final:
    pac_man_com_busca(nodo_final, iterations, False)
    print(f"m: {m} - Solution found; Iterations: {iterations}")
    break
  else:
    print(f"m: {m} - No solution found; Iterations: {iterations}")

m: 0 - No solution found; Iterations: 1
m: 1 - No solution found; Iterations: 3
m: 2 - No solution found; Iterations: 6
m: 3 - No solution found; Iterations: 11
m: 4 - No solution found; Iterations: 18
m: 5 - No solution found; Iterations: 28
m: 6 - No solution found; Iterations: 39
m: 7 - No solution found; Iterations: 55
m: 8 - No solution found; Iterations: 73
Number of iterations: 98

m: 9 - Solution found; Iterations: 98


## Teste com todos os campos com poda

In [25]:
iterações = []

for index, field in enumerate(fields):
  nodo_final, iterations = busca_em_profundidade(Node(field, None, 0, 0), 200, True)
  iterações.append(iterations)
  print(f"- Field {index + 1}: {iterations} iterations")

- Field 1: 98 iterations
- Field 2: 1389 iterations
- Field 3: 11367 iterations
- Field 4: 15760 iterations


# Busca bidirecional

In [ ]:
def busca_bidirecional(initial_node, meta_node):
    iterations = 0
    current_nodes = [initial_node]
    current_nodes_meta = [meta_node]

    visited = set()
    visited_meta = set()

    while current_nodes and current_nodes_meta:
        current = current_nodes.pop(0)
        state_key = current.get_state_key()
        if state_key in visited_meta:
            return current, iterations
        visited.add(state_key)

        for neighbor in current.generate_next_nodes(prune=True):
            key = neighbor.get_state_key()
            if key not in visited:
                current_nodes.append(neighbor)

        current_meta = current_nodes_meta.pop(0)
        state_key_meta = current_meta.get_state_key()
        if state_key_meta in visited:
            return current_meta, iterations
        visited_meta.add(state_key_meta)

        for neighbor in current_meta.generate_backtrack_nodes(current.field, prune=True):
            key = neighbor.get_state_key()
            if key not in visited_meta:
                current_nodes_meta.append(neighbor)

        iterations += 1

    return None, iterations


In [5]:
field_easy_done = [
  [0, 0, 0, 0],
  [0, 2, 2, 0],
  [0, 0, 2, 0],
  [0, 0, 0, 0],
]

busca_bidirecional(Node(field_easy, None, 0, 0), Node(field_easy_done, None, 2, 1))

(None, 649)

# Busca A*